# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# Libraries for data handling and storage
import pandas as pd
from sqlalchemy import create_engine
import pickle

# Text processing libraries
import re
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Machine learning libraries
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

# Classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Others
import warnings


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Messages_table_v1', engine)

# Define feature and target variables X and Y
# 'message' is my feature
X = df['message']  

# ignore columns 'id','message','original','genre' so all other columns thereafter are my target
Y = df.iloc[:, 4:] 

In [3]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """
    Tokenizes text data

    Args:
    text str: Messages as text data

    Returns:
    clean_tokens list: Processed text after normalizing, tokenizing and lemmatizing
    """
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    words = word_tokenize(text)
    
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    
    # Reduce words to their root form
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(w) for w in words]
    
    # Lemmatize verbs by specifying pos
    clean_tokens = [lemmatizer.lemmatize(w, pos='v') for w in lemmatized]
    
    return clean_tokens

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Peter.Derkx\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Peter.Derkx\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Peter.Derkx\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Peter.Derkx\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

In [7]:
pipeline

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000269FB21DA80>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
# split data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

# train classifier
pipeline.fit(X_train, Y_train)

c:\Users\Peter.Derkx\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000269FB21DA80>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [23]:
# Predict on test data:
Y_pred = pipeline.predict(X_test)

# List to store the results for each category
reports = []

# Loop through each category
for i, col in enumerate(Y_test.columns):
    
    # Get the classification report for each column
    report = classification_report(Y_test[col], Y_pred[:, i], zero_division=1, output_dict=True)
    
    # Extracting the metrics of interest
    precision_0 = report['0']['precision'] if '0' in report else None
    recall_0 = report['0']['recall'] if '0' in report else None
    f1_0 = report['0']['f1-score'] if '0' in report else None

    precision_1 = report['1']['precision'] if '1' in report else None
    recall_1 = report['1']['recall'] if '1' in report else None
    f1_1 = report['1']['f1-score'] if '1' in report else None

    weighted_avg_precision = report['weighted avg']['precision']
    weighted_avg_recall = report['weighted avg']['recall']
    weighted_avg_f1 = report['weighted avg']['f1-score']

    # Store the results in a dictionary
    reports.append({
        'Category': col,
        'Precision_0': precision_0,
        'Recall_0': recall_0,
        'F1_0': f1_0,
        'Precision_1': precision_1,
        'Recall_1': recall_1,
        'F1_1': f1_1,
        'Weighted_Avg_Precision': weighted_avg_precision,
        'Weighted_Avg_Recall': weighted_avg_recall,
        'Weighted_Avg_F1': weighted_avg_f1
    })

# Convert the results into a DataFrame
df_reports = pd.DataFrame(reports)

# Set precision for the entire DataFrame
pd.options.display.float_format = "{:.2f}".format

# Apply the heatmap styling
styled_df = df_reports.style.background_gradient(cmap='coolwarm', axis=None)

display(styled_df)


,Category,Precision_0,Recall_0,F1_0,Precision_1,Recall_1,F1_1,Weighted_Avg_Precision,Weighted_Avg_Recall,Weighted_Avg_F1
0,related,0.589189,0.348688,0.438103,0.812948,0.916869,0.861787,0.755515,0.776778,0.756431
1,request,0.843113,0.995223,0.912875,0.798450,0.092709,0.166129,0.835542,0.842234,0.786290
2,offer,0.994964,0.999847,0.997399,0.000000,0.000000,0.000000,0.989954,0.994812,0.992377
3,aid_related,0.600094,0.986354,0.746202,0.688235,0.043820,0.082394,0.636001,0.602380,0.475777
4,medical_help,0.918486,0.999668,0.957359,0.333333,0.001869,0.003717,0.870720,0.918218,0.879513
5,medical_products,0.947786,0.999678,0.973041,0.500000,0.005814,0.011494,0.924283,0.947513,0.922572
6,search_and_rescue,0.975740,1.000000,0.987721,1.000000,0.000000,0.000000,0.976329,0.975740,0.963759
7,security,0.982301,1.000000,0.991071,1.000000,0.000000,0.000000,0.982614,0.982301,0.973530
8,military,0.969484,1.000000,0.984506,1.000000,0.000000,0.000000,0.970415,0.969484,0.954463
9,child_alone,1.000000,1.000000,1.000000,nan,nan,nan,1.000000,1.000000,1.000000


High Precision, Low Recall Categories:

For several categories like offer, search_and_rescue, security, military, missing_people, refugees, death, transport, electricity, tools, hospitals, shops, aid_centers, and more, the Precision for class 1 is high or perfect (equal to 1), but the Recall is very low or 0. This indicates that while the model makes very few false positives for these categories, it also misses a lot of true positives. 

### 6. Improve your model
Use grid search to find better parameters. 

In [28]:
# Suppress all warnings
warnings.simplefilter("ignore")

# Define parameters for GridSearch
parameters = {
    # 'vect__max_df': [0.5, 0.75, 1.0],  # max_df: remove terms that have a document frequency strictly higher than the threshold
    'vect__ngram_range': [(1, 1), (1, 2)],  # unigrams or bigrams
    # 'vect__stop_words': ['english', None],  # use English stop words or not
    # 'tfidf__use_idf': [True, False],  # use inverse-document-frequency reweighting or not
    'clf__estimator__n_neighbors': [7, 9],  # number of neighbors for KNN
    # 'clf__estimator__weights': ['uniform', 'distance']  # weight function used in prediction
    # ... 
}

# Initialize GridSearch, limit cross validation to 3 fold to speed up gridsearch
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, cv=3)

# Fit the GridSearch model
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 1/3] END clf__estimator__n_neighbors=7, vect__ngram_range=(1, 1);, score=0.248 total time= 4.0min
[CV 2/3] END clf__estimator__n_neighbors=7, vect__ngram_range=(1, 1);, score=0.251 total time= 4.1min
[CV 3/3] END clf__estimator__n_neighbors=7, vect__ngram_range=(1, 1);, score=0.255 total time= 4.1min
[CV 1/3] END clf__estimator__n_neighbors=7, vect__ngram_range=(1, 2);, score=0.242 total time= 8.8min
[CV 2/3] END clf__estimator__n_neighbors=7, vect__ngram_range=(1, 2);, score=0.251 total time= 7.8min
[CV 3/3] END clf__estimator__n_neighbors=7, vect__ngram_range=(1, 2);, score=0.251 total time= 7.8min
[CV 1/3] END clf__estimator__n_neighbors=9, vect__ngram_range=(1, 1);, score=0.244 total time= 6.1min
[CV 2/3] END clf__estimator__n_neighbors=9, vect__ngram_range=(1, 1);, score=0.250 total time= 6.0min
[CV 3/3] END clf__estimator__n_neighbors=9, vect__ngram_range=(1, 1);, score=0.252 total time= 4.0min
[CV 1/3] END clf__esti

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000269FB21DA80>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=KNeighborsClassifier()))]),
             param_grid={'clf__estimator__n_neighbors': [7, 9],
                         'vect__ngram_range': [(1, 1), (1, 2)]},
             verbose=3)

In [29]:
# Print the best parameters found by GridSearchCV
print(cv.best_params_)

{'clf__estimator__n_neighbors': 7, 'vect__ngram_range': (1, 1)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [30]:
# 'cv' is my GridSearchCV instance
best_pipeline = cv.best_estimator_

# Predict on test data using the improved model:
Y_pred_improved = best_pipeline.predict(X_test)

# List to store the results for each category
reports_improved = []

# Loop through each category
for i, col in enumerate(Y_test.columns):
    
    # Get the classification report for each column
    report = classification_report(Y_test[col], Y_pred_improved[:, i], zero_division=1, output_dict=True)
    
    # Extracting the metrics of interest
    precision_0 = report['0']['precision'] if '0' in report else None
    recall_0 = report['0']['recall'] if '0' in report else None
    f1_0 = report['0']['f1-score'] if '0' in report else None

    precision_1 = report['1']['precision'] if '1' in report else None
    recall_1 = report['1']['recall'] if '1' in report else None
    f1_1 = report['1']['f1-score'] if '1' in report else None

    weighted_avg_precision = report['weighted avg']['precision']
    weighted_avg_recall = report['weighted avg']['recall']
    weighted_avg_f1 = report['weighted avg']['f1-score']

    # Store the results in a dictionary
    reports_improved.append({
        'Category': col,
        'Precision_0': precision_0,
        'Recall_0': recall_0,
        'F1_0': f1_0,
        'Precision_1': precision_1,
        'Recall_1': recall_1,
        'F1_1': f1_1,
        'Weighted_Avg_Precision': weighted_avg_precision,
        'Weighted_Avg_Recall': weighted_avg_recall,
        'Weighted_Avg_F1': weighted_avg_f1
    })

# Convert the results into a DataFrame
df_reports_improved = pd.DataFrame(reports_improved)

# Set precision for the entire DataFrame
pd.options.display.float_format = "{:.2f}".format

# Apply the heatmap styling
styled_df_improved = df_reports_improved.style.background_gradient(cmap='coolwarm', axis=None)

display(styled_df_improved)

,Category,Precision_0,Recall_0,F1_0,Precision_1,Recall_1,F1_1,Weighted_Avg_Precision,Weighted_Avg_Recall,Weighted_Avg_F1
0,related,0.582809,0.355726,0.441796,0.813699,0.917880,0.862656,0.756855,0.779219,0.758766
1,request,0.858737,0.989528,0.919505,0.797872,0.202520,0.323044,0.848420,0.856118,0.818396
2,offer,0.994965,1.000000,0.997476,1.000000,0.000000,0.000000,0.994990,0.994965,0.992454
3,aid_related,0.630511,0.968332,0.763732,0.791171,0.174532,0.285977,0.695962,0.644950,0.569102
4,medical_help,0.919303,0.999335,0.957650,0.636364,0.013084,0.025641,0.896207,0.918828,0.881571
5,medical_products,0.949503,0.999195,0.973715,0.736842,0.040698,0.077135,0.938341,0.948886,0.926656
6,search_and_rescue,0.976038,1.000000,0.987874,1.000000,0.012579,0.024845,0.976619,0.976045,0.964511
7,security,0.982301,1.000000,0.991071,1.000000,0.000000,0.000000,0.982614,0.982301,0.973530
8,military,0.969919,0.999685,0.984577,0.600000,0.015000,0.029268,0.958631,0.969637,0.955425
9,child_alone,1.000000,1.000000,1.000000,nan,nan,nan,1.000000,1.000000,1.000000


The precision, recall, and F1 score for class 1 (Precision_1, Recall_1, F1_1) seems to have generally increased, indicating improved performance in identifying true positives. Also the Weighted_Avg_Precision, Weighted_Avg_Recall, and Weighted_Avg_F1 seem to have increased for most categories, indicating better overall performance.
However, there are still certain categories where the performance (especially for the positive class) could benefit from further optimization.

For this considered 'rescue center', I assume recall is often prioritized to ensure no one in need is overlooked. However, high precision is also essential to prevent straining resources with false alarms. The relative importance can vary based on the specific situation and available resources. Ideally, a balance between recall and precision ensures efficient resource allocation without missing genuine emergencies.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [31]:
from sklearn.ensemble import RandomForestClassifier
# use the RandomForestClassifier instead of KNeighborsClassifier

# Suppress all warnings
warnings.simplefilter("ignore")

# Define pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

# Parameters for grid search. Other features:
# Regularization by increase minimum number of samples required at a leaf node to prevent overfitting
parameters = {
    'clf__estimator__min_samples_leaf': [1, 2]     
}

# Initialize GridSearch, limit cross validation to 3 fold to speed up gridsearch
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, cv=3)

# Fit the GridSearch model
cv.fit(X_train, Y_train)

# Print the best parameters found by GridSearchCV
print(cv.best_params_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV 1/3] END clf__estimator__min_samples_leaf=1;, score=0.258 total time=16.6min
[CV 2/3] END clf__estimator__min_samples_leaf=1;, score=0.267 total time=18.6min
[CV 3/3] END clf__estimator__min_samples_leaf=1;, score=0.266 total time=18.3min
[CV 1/3] END clf__estimator__min_samples_leaf=2;, score=0.226 total time= 8.7min
[CV 2/3] END clf__estimator__min_samples_leaf=2;, score=0.232 total time= 7.6min
[CV 3/3] END clf__estimator__min_samples_leaf=2;, score=0.232 total time= 7.9min
{'clf__estimator__min_samples_leaf': 1}


In [32]:
# 'cv' is my GridSearchCV instance
best_pipeline = cv.best_estimator_

# Predict on test data using the improved model:
Y_pred_improved = best_pipeline.predict(X_test)

# List to store the results for each category
reports_improved = []

# Loop through each category
for i, col in enumerate(Y_test.columns):
    
    # Get the classification report for each column
    report = classification_report(Y_test[col], Y_pred_improved[:, i], zero_division=1, output_dict=True)
    
    # Extracting the metrics of interest
    precision_0 = report['0']['precision'] if '0' in report else None
    recall_0 = report['0']['recall'] if '0' in report else None
    f1_0 = report['0']['f1-score'] if '0' in report else None

    precision_1 = report['1']['precision'] if '1' in report else None
    recall_1 = report['1']['recall'] if '1' in report else None
    f1_1 = report['1']['f1-score'] if '1' in report else None

    weighted_avg_precision = report['weighted avg']['precision']
    weighted_avg_recall = report['weighted avg']['recall']
    weighted_avg_f1 = report['weighted avg']['f1-score']

    # Store the results in a dictionary
    reports_improved.append({
        'Category': col,
        'Precision_0': precision_0,
        'Recall_0': recall_0,
        'F1_0': f1_0,
        'Precision_1': precision_1,
        'Recall_1': recall_1,
        'F1_1': f1_1,
        'Weighted_Avg_Precision': weighted_avg_precision,
        'Weighted_Avg_Recall': weighted_avg_recall,
        'Weighted_Avg_F1': weighted_avg_f1
    })

# Convert the results into a DataFrame
df_reports_improved = pd.DataFrame(reports_improved)

# Set precision for the entire DataFrame
pd.options.display.float_format = "{:.2f}".format

# Apply the heatmap styling
styled_df_improved = df_reports_improved.style.background_gradient(cmap='coolwarm', axis=None)

display(styled_df_improved)

,Category,Precision_0,Recall_0,F1_0,Precision_1,Recall_1,F1_1,Weighted_Avg_Precision,Weighted_Avg_Recall,Weighted_Avg_F1
0,related,0.707514,0.391555,0.504119,0.828288,0.946400,0.883414,0.796594,0.809887,0.789427
1,request,0.902909,0.980709,0.940203,0.836449,0.483348,0.612664,0.891643,0.896399,0.884680
2,offer,0.994965,1.000000,0.997476,1.000000,0.000000,0.000000,0.994990,0.994965,0.992454
3,aid_related,0.796382,0.838826,0.817053,0.745838,0.688015,0.715761,0.775791,0.777388,0.775788
4,medical_help,0.925355,0.996843,0.959770,0.728571,0.095327,0.168595,0.909291,0.923253,0.895186
5,medical_products,0.951541,0.999195,0.974786,0.848485,0.081395,0.148541,0.946132,0.951022,0.931419
6,search_and_rescue,0.976762,0.999062,0.987786,0.538462,0.044025,0.081395,0.966129,0.975893,0.965797
7,security,0.982443,0.999534,0.990915,0.250000,0.008621,0.016667,0.969479,0.981996,0.973671
8,military,0.971525,0.998741,0.984945,0.636364,0.070000,0.126126,0.961297,0.970400,0.958737
9,child_alone,1.000000,1.000000,1.000000,nan,nan,nan,1.000000,1.000000,1.000000


significant delta in recall_1:
aid_related +0.51, food +0.47, shelter +0.30, water +0.29, request+0.28 

The provided data showcases notable improvements in the Recall_1 scores for key rescue service categories. The 'aid_related' category saw the highest gain, signifying better recognition of general aid needs. Critical essentials like 'food', 'shelter', and 'water' also witnessed significant enhancements. The 'request' category's boost indicates heightened sensitivity to calls for help. Overall, the new system demonstrates heightened efficiency in identifying rescue needs.

### 9. Export your model as a pickle file

In [34]:
import pickle

# Save to file
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.